# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,coracora,-15.0333,-73.7833,6.65,31,1,0.97,PE,1723177861
1,1,menaka,15.9182,2.4022,29.62,58,68,5.61,ML,1723177862
2,2,nandikotkur,15.8667,78.2667,29.58,65,91,5.28,IN,1723177863
3,3,hasaki,35.7333,140.8333,28.99,79,2,7.11,JP,1723177623
4,4,vangaindrano,-23.3500,47.6000,20.99,88,64,2.54,MG,1723177867


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_data_df['coords'] = list(zip(city_data_df['Lat'], city_data_df['Lng']))

# Plotting the map with coordinates
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    size=10,
    tiles='OSM',  
    color='City',  
    hover_cols=['City', 'Country', 'Max Temp', 'Humidity']
)

# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Max Temp,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
criteria = ( (city_data_df["Max Temp"] >= 25) &
    (city_data_df["Max Temp"] <= 35) &
    (city_data_df["Humidity"] >= 80)
)

# Apply the criteria
filtered_cities_df = city_data_df[criteria]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Drop the 'coords' column from the filtered DataFrame
filtered_cities_df = filtered_cities_df.drop(columns=['coords'])

# Display sample data
filtered_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,13,amanganj,24.4333,80.0333,25.11,89,100,2.67,IN,1723177886
68,68,acaponeta,22.4964,-105.3594,26.44,93,81,1.36,MX,1723177985
80,80,utrik,11.2278,169.8474,28.40,80,86,7.19,MH,1723177606
93,93,arawa,-6.2132,155.5367,27.90,87,90,1.63,PG,1723178032
110,110,kailua-kona,19.6406,-155.9956,28.98,90,40,4.63,US,1723177597
124,124,al ghayzah,16.2079,52.1760,26.07,82,43,3.01,YE,1723178090
128,128,twon-brass,4.3151,6.2418,25.01,84,100,3.64,NG,1723178099
156,156,sterling,39.0062,-77.4286,26.16,87,100,5.66,US,1723177942
167,167,acapulco de juarez,16.8634,-99.8901,25.90,83,75,5.14,MX,1723178169
186,186,ambikapur,23.1167,83.2000,26.76,80,100,2.97,IN,1723178202


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
13,amanganj,IN,24.4333,80.0333,89,
68,acaponeta,MX,22.4964,-105.3594,93,
80,utrik,MH,11.2278,169.8474,80,
93,arawa,PG,-6.2132,155.5367,87,
110,kailua-kona,US,19.6406,-155.9956,90,
124,al ghayzah,YE,16.2079,52.1760,82,
128,twon-brass,NG,4.3151,6.2418,84,
156,sterling,US,39.0062,-77.4286,87,
167,acapulco de juarez,MX,16.8634,-99.8901,83,
186,ambikapur,IN,23.1167,83.2000,80,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
   "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
amanganj - nearest hotel: No hotel found
acaponeta - nearest hotel: Hotel Quinta Blanca
utrik - nearest hotel: No hotel found
arawa - nearest hotel: Bovotel
kailua-kona - nearest hotel: Kona Seaside Hotel
al ghayzah - nearest hotel: فندق تاج العرب
twon-brass - nearest hotel: KC Hotel
sterling - nearest hotel: Residence Inn Dulles Airport at Dulles 28 Centre
acapulco de juarez - nearest hotel: Hotel del Valle
ambikapur - nearest hotel: No hotel found
tual - nearest hotel: Suita
weno - nearest hotel: High Tide Hotel
santiago de la pena - nearest hotel: Sara
san luis de la loma - nearest hotel: No hotel found
bull savanna - nearest hotel: Junction Guest House
faya-largeau - nearest hotel: No hotel found
victoria - nearest hotel: 四季酒店 Four Seasons Hotel
nova sintra - nearest hotel: Residência Ka Dencho
kavaratti - nearest hotel: No hotel found
selibaby - nearest hotel: Hotel Wagadu
puerto el triunfo - nearest hotel: Motel Jardín
lerdo de tejada - nearest hotel: No hot

,City,Country,Lat,Lng,Humidity,Hotel Name
13,amanganj,IN,24.4333,80.0333,89,No hotel found
68,acaponeta,MX,22.4964,-105.3594,93,Hotel Quinta Blanca
80,utrik,MH,11.2278,169.8474,80,No hotel found
93,arawa,PG,-6.2132,155.5367,87,Bovotel
110,kailua-kona,US,19.6406,-155.9956,90,Kona Seaside Hotel
124,al ghayzah,YE,16.2079,52.1760,82,فندق تاج العرب
128,twon-brass,NG,4.3151,6.2418,84,KC Hotel
156,sterling,US,39.0062,-77.4286,87,Residence Inn Dulles Airport at Dulles 28 Centre
167,acapulco de juarez,MX,16.8634,-99.8901,83,Hotel del Valle
186,ambikapur,IN,23.1167,83.2000,80,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot
hotel_df['coords'] = list(zip(hotel_df['Lat'], hotel_df['Lng']))

# Plotting the map with coordinates
hotel_map_plot = hotel_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    size=10,
    tiles='OSM',  
    color='Hotel Name',  
    hover_cols=['City', 'Humidity', 'Hotel Name', 'Country']
)

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,City,Humidity,Country)